## Zadanie 10

In [2]:
import csv
from collections import defaultdict
import scipy.stats
data = defaultdict(list)

with open('rp11-10.csv', newline='') as csvfile:
    tmp = {1:'Cukier',2:'Waga',3:'Cisnienie'}
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if (row[0] != "Cukier"):
            for i in range(len(row)):
                data[tmp[i + 1]].append(int(row[i]))

print(data)

defaultdict(<class 'list'>, {'Cukier': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 'Waga': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], 'Cisnienie': [70, 80, 83, 85, 65, 85, 80, 90, 95, 88, 85, 90, 95, 90, 86, 90, 95, 100, 105, 110]})


#### część 1 - hipoteza o równości średnich dla grup o podwyższonym cukrze i w normie
jeśli średnie się różnią to ciśnienie zależy od cukru

In [3]:
groups = defaultdict(list)
general_avg = 0
avgs = defaultdict(int)

for i in range(len(data["Cisnienie"])):
    if data['Cukier'][i] == 1:
        groups[1].append(data["Cisnienie"][i])
    else:
        groups[2].append(data["Cisnienie"][i])
    general_avg += data["Cisnienie"][i]
    
general_avg /= 2 * len(groups[1])

avgs[1] = sum(groups[1]) / len(groups[1])
avgs[2] = sum(groups[2]) / len(groups[2])

SSA = len(groups[1]) * ((avgs[1] - general_avg)**2 + (avgs[2] - general_avg)**2)
SSTot = 0
for i in groups.keys():
    for j in groups[i]:
        SSTot += (j - general_avg)**2
        
SSE = SSTot - SSA

MSA = SSA / 1 #I - 1  = (2 - 1)
MSE = SSE / 18 # I*(n -1) = (2 * (9 - 1))

t = MSA / MSE

p_value = 1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 18)
print(p_value)

# Na poziomie istotności alfa hipotezę odrzucamy (czyli cukier ma wpływ na ciśnienie)

0.004468622627937235


#### część 2 - hipoteza o równości średnich dla grup o podwyższonej wadze i normalnej
jeśli średnie się różnią to ciśnienie zależy od wagi

In [4]:
groups = defaultdict(list)
general_avg = 0
avgs = defaultdict(int)

for i in range(len(data["Cisnienie"])):
    if data['Waga'][i] == 1:
        groups[1].append(data["Cisnienie"][i])
    else:
        groups[2].append(data["Cisnienie"][i])
    general_avg += data["Cisnienie"][i]
    
general_avg /= 2 * len(groups[1])

avgs[1] = sum(groups[1]) / len(groups[1])
avgs[2] = sum(groups[2]) / len(groups[2])

SSB = len(groups[1]) * ((avgs[1] - general_avg)**2 + (avgs[2] - general_avg)**2)
SSTot = 0
for i in groups.keys():
    for j in groups[i]:
        SSTot += (j - general_avg)**2
        
SSE = SSTot - SSB

MSB = SSB / 1 #I - 1  = (2 - 1)
MSE = SSE / 18 # I*(n -1) = (2 * (9 - 1))

t = MSB / MSE

p_value = 1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 18)
print(p_value)

# Na poziomie istotności alfa hipotezę odrzucamy (czyli waga ma wpływ na ciśnienie)

0.016224146737865652


#### część 3 - ANOVA 2 czynnikowa z powtórzeniami - czy istnieje interakcja wagi z cukrem
hipoteza: istnieje

In [5]:
groups = defaultdict(lambda: defaultdict(list)) #cukier w normie i zawyżony

for i in range(len(data['Cisnienie'])):
    if data['Cukier'][i] == 1:
        groups[1][data['Waga'][i]].append(data['Cisnienie'][i])
    else:
        groups[2][data['Waga'][i]].append(data['Cisnienie'][i])

SSE = 0
for i in [1,2]:
    for j in [1,2]:
        avg_in_observ = sum(groups[i][j])/len(groups[i][j])
        for k in range(len(groups[i][j])):
            SSE += (groups[i][j][k] - avg_in_observ)**2


SSAB = SSTot-SSA-SSB-SSE
MSE = SSE/16 # I*J*K - I*J = 2*2*5 - 2*2

MSAB = SSAB/1 # (I-1)(J-1) = (2-1) * (2-1)

t = MSAB/MSE
p_value = 1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 16)
print(p_value)
#p_value bardzo duże, przy poziomie istotności alfa = 0.05 hipotezy nie odrzucamy


0.9741631598589974


### Zadanie 12

#### hipoteza o równości średnich przed i po diecie

In [6]:
data = defaultdict(list)
global_avg = 0
with open('rp11-12.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        for i in range(len(row)):
            data[i].append(int(row[i]))
            global_avg += int(row[i])
global_avg /= len(data[0]) * 2
print(data)

defaultdict(<class 'list'>, {0: [93, 99, 95, 80, 86, 98, 98, 94, 89, 95, 97, 96, 90, 85, 95, 91], 1: [88, 95, 86, 77, 88, 94, 94, 88, 93, 95, 91, 82, 90, 79, 81, 89]})


In [7]:
avgs = defaultdict(list)

avgs[0] = sum(data[0]) / len(data[0])
avgs[1] = sum(data[1]) / len(data[1])

SSA = 16 * ((avgs[0] - global_avg)**2 + (avgs[1] - global_avg)**2)

SSTot = 0
for key in data.keys():
    for i in range(len(data[key])):
        SSTot += (data[key][i] - global_avg)**2

SSE = SSTot - SSA

MSA = SSA / 1 # I - 1 = 2 - 1
MSE = SSE / 30 # I(n - 1) = 2(16 - 1)

t = MSA / MSE

p_value = 1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 30)
print(p_value)

#Na poziomie istotności alfa = 0.05 odrzucamy hipotezę


0.03178374262318373


### Zadanie 13

#### Hipoteza o równości średnich dla dówch typów uprawy

In [10]:
data = defaultdict(list)
global_avg = 0
with open('rp11-13.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        for i in range(len(row)):
            data[i].append(int(row[i]))
            global_avg += int(row[i])
global_avg /= len(data[0]) * 2
#print(data)


In [11]:
avgs = defaultdict(list)

avgs[0] = sum(data[0]) / len(data[0])
avgs[1] = sum(data[1]) / len(data[1])


SSA = 10 * ((avgs[0] - global_avg)**2 + (avgs[1] - global_avg)**2)

SSTot  = 0
for key in data.keys():
    for i in range(len(data[key])):
        SSTot += (data[key][i] - global_avg)**2

SSE = SSTot - SSA

MSA = SSA / 1 # I - 1 = 2 - 1
MSE = SSE / 18 # I(n - 1) = 2 (10 - 1)

t = MSA/MSE

p_value = 1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 18)
#p_value = min(1 - scipy.stats.f.cdf(t, dfn = 1, dfd = 18),scipy.stats.f.cdf(t, dfn = 1, dfd = 18))
print(p_value)
# Na poziomie istotności alfa = 0.05 nie ma podstaw do odrzucenia hipotezy

0.545400427864513
